In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
pip install -i https://test.pypi.org/simple/ my_krml_25076922==2024.0.3.3

Looking in indexes: https://test.pypi.org/simple/
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import numpy as np

import os

/Users/vega7unk/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [4]:
path ='/Users/vega7unk/Documents/3rd_Sem_DSI/Adv_ML/AT2/Project/adv_mla_at2/data'

sales_data = pd.read_csv(f'{path}/df_transformed.csv')

In [5]:
from my_krml_25076922.features.data_processing import scale_features

# List of features to scale
features_to_scale = ['total_sales', 'day_of_week', 'month','year']

# Apply scaling to the numerical columns
sales_data = scale_features(sales_data, features_to_scale)

# Preview the scaled data
sales_data.head()

,item_id,dept_id,cat_id,store_id,state_id,total_sales,day_of_week,month,year
0,1437,3,1,0,0,-0.650955,0.993873,-1.539218,-1.371861
1,1438,3,1,0,0,-0.650955,0.993873,-1.539218,-1.371861
2,1439,3,1,0,0,-0.650955,0.993873,-1.539218,-1.371861
3,1440,3,1,0,0,-0.650955,0.993873,-1.539218,-1.371861
4,1441,3,1,0,0,-0.650955,0.993873,-1.539218,-1.371861


In [6]:
target = sales_data.pop('total_sales')

In [8]:
from my_krml_25076922.data.sets import split_sets_random

X_train, y_train, X_val, y_val, X_test, y_test = split_sets_random(sales_data, target, test_ratio=0.2)

In [9]:
from my_krml_25076922.models.null import NullRegressor

base_model = NullRegressor()

In [10]:
y_base = base_model.fit_predict(y_train)

In [11]:
from my_krml_25076922.models.performance import print_regressor_scores

print_regressor_scores(y_preds=y_base, y_actuals=y_train, set_name='Training')

RMSE Training: 1.0000917739425756
MAE Training: 0.9145173942251014
